In [1]:
from dash import Dash, dash_table, dcc, html
from dash.dependencies import Input, Output
import pandas as pd
from flask import Flask
from Librerias_SAP import SAP_GUI, Funtions
import pandas as pd
import re
from collections import OrderedDict
from dash import dash_table as dt
from dash import dcc
from dash import html
from dash.dependencies import Input
from dash.dependencies import Output
import numpy as np
import dash_bootstrap_components as dbc
from datetime import datetime, timedelta

In [2]:
"""----------------------------Inciar session----------------------------------------------------"""
# Insert User name and password

Keys=pd.read_excel("C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Claves\\Keys.xlsx")
user=Keys["User"][0]
password=Keys["Password"][0]
# Initialize session
session=SAP_GUI.SessionSAP(user,password)

In [3]:
Defaul_Column_Pedidos_dia=['Documento', 'GTr', 'Denomin.', 'ClVt', 'Denominación', 'Solic.',
       'Creado el', 'Fecha doc.', 'Pedido', 'Func.', 'Responsab', 'Creado',
       'OrgVt', 'CDis', 'Se', 'OfVta', 'GVen', 'Mon.', 'Valor neto']

In [4]:
# PEDIDOSEXITO
Transsaccion='va05n'
provision='PEDIDOSEXITO'
variant='JESPINOSAP'

SAP_GUI.Search_VA05N(Transsaccion,session,provision,variant)

<COMObject <unknown>>

In [5]:
Name_VA05N="Pedidos_diarios"
Ruta_Exito_ZSD010_ZVMI="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\"
SAP_GUI.Export_TXT2(Name_VA05N,session,Ruta_Exito_ZSD010_ZVMI)

In [6]:
Pedidos_VN05N=pd.read_csv(Ruta_Exito_ZSD010_ZVMI+Name_VA05N+".txt",delimiter="\t",skiprows=1)
Pedidos_VN05N=Funtions.Clean_Columns(Pedidos_VN05N)
Pedidos_VN05N=Funtions.default_column(Defaul_Column_Pedidos_dia,Pedidos_VN05N)
#Elimino pedidos con valores nulos
Pedidos_VN05N=Pedidos_VN05N[Pedidos_VN05N['Pedido'].notnull()]

In [9]:
# Estandarizo

Agenda=["85","20","146","149","50","138","45"]

Pedidos_VN05N['Pedido']=Funtions.Estandarizo_Pedidos(Pedidos_VN05N['Pedido'])
Pedidos_VN05N['Pedido']=Funtions.complete_pedidos(Pedidos_VN05N['Pedido'],Agenda)

# Condicion especial para el filtro por cliente

In [46]:
#Clientes lunes
Lunes_Cliente_Exito=["0085","0045"]  #"0085"  Funza, Surtimayoristas
Lunes_Cliente_Cencosub=["93-","122-","127-","95-"] #"93-","122-","127-" Medellin, Barranquilla, Bucaramanga y cali
#Clientes Martes
Martes_Cliente_Exito=["0020","0045"]  #"0020"  VEGAS, Surtimayoristas
Martes_Cliente_Cencosub=["93-","122-","127-"] #"93-","122-","127-" Medellin, Barranquilla y Bucaramanga
#Clientes Miercoles
Miecoles_Cliente_Exito=["0020","0045"]  #"0085"  Funza
#Clientes Jueves
Jueves_Cliente_Exito=["0020","0146","0149","0045"]  #"0020"  VEGAS, Barranquilla, Bucaramanga, Surtimayoristas
Jueves_Cliente_Cencosub=["93-","122-","127-"] #"93-","122-","127-" Medellin, Barranquilla y Bucaramanga
#Clientes Jueves
Viernes_Cliente_Exito=["0050","0138","0045"]  #"0020"  Cali, Pereira, Surtimayoristas 
Viernes_Cliente_Cencosub=["60-"] #"60" Bogota

In [13]:
Lunes=["0085"]

buscar=Funtions.Search_Agenda_Exito(Pedidos_VN05N['Pedido'],Lunes)
filtro_cencosub=Pedidos_VN05N[Pedidos_VN05N['Pedido'].isin(buscar)]

In [14]:
filtro_cencosub['Pedido'].drop_duplicates().value_counts()

0085761010    1
0085768263    1
0085768267    1
0085768261    1
0085768259    1
             ..
0085764471    1
0085764470    1
0085764463    1
0085764490    1
0085770620    1
Name: Pedido, Length: 145, dtype: int64

In [16]:
#Busco los que tengan Pedidos, Entregados, Despachados y facturados. 
Tabla_ZSD79=SAP_GUI.Search_ZSD79('zsd79',filtro_cencosub['Pedido'].drop_duplicates(),session)

In [17]:
table=Tabla_ZSD79

# Nota no tiene entrega generada si Pedido Concluido Rojo y entrega conlcuida Verda

In [45]:
def Search_Table_ZSD79(table,session):
    """
    -table: Tabla ZSD79
    -session: session
    (Busca filtro especial de colores)
    """
    #Columna Pedido concluido 
    table.SelectColumn("LFGSK")
    #Columna entregas concluidas
    table.SelectColumn("WBSTK")
    session.findById("wnd[0]/mbar/menu[1]/menu[3]").select() 
    session.findById("wnd[1]/usr/ssub%_SUBSCREEN_FREESEL:SAPLSSEL:1105/ctxt%%DYN001-LOW").text = "@0A@"  #Rojo Pedido
    session.findById("wnd[1]/usr/ssub%_SUBSCREEN_FREESEL:SAPLSSEL:1105/ctxt%%DYN002-LOW").text = "@08@"  #Verde Entrega
    session.findById("wnd[1]/tbar[0]/btn[0]").press()
    Row=table.RowCount
    dic={"Nº Pedido cliente":[],"Clase Orden":[],"PrimFecEnt":[],"ÚltEntrega":[]}
    for i in range(0,Row):
        #Nº Pedido cliente Col "BSTNK"
        #Clase Orden Col "AUART"
        #PrimFecEnt Col "AUDAT"
        #ÚltEntrega Col "VDATU"
        Pedido_cliente,Clase_Orden,PrimFecEnt,ÚltEntrega=table.GetCellValue(i,"BSTNK"),table.GetCellValue(2,"AUART"),table.GetCellValue(2,"AUDAT"),table.GetCellValue(2,"VDATU") 
        dic["Nº Pedido cliente"].append(Pedido_cliente)
        dic["Clase Orden"].append(Clase_Orden)
        dic["PrimFecEnt"].append(PrimFecEnt)
        dic["ÚltEntrega"].append(ÚltEntrega)
    return(pd.DataFrame(dic))

In [44]:
#Buscar los pedidos diarios de clientes
Search_Table_ZSD79(Tabla_ZSD79,session)

,Nº Pedido cliente,Clase Orden,PrimFecEnt,ÚltEntrega
0,0085770424,ZVMI,29.06.2023,06.07.2023
1,0085770425,ZVMI,29.06.2023,06.07.2023
2,0085770426,ZVMI,29.06.2023,06.07.2023
3,0085770427,ZVMI,29.06.2023,06.07.2023
4,0085770428,ZVMI,29.06.2023,06.07.2023
5,0085770429,ZVMI,29.06.2023,06.07.2023
6,0085770430,ZVMI,29.06.2023,06.07.2023
7,0085770431,ZVMI,29.06.2023,06.07.2023
8,0085770432,ZVMI,29.06.2023,06.07.2023
9,0085770433,ZVMI,29.06.2023,06.07.2023
